In [1]:
import lzma
import pickle
from pathlib import Path

import dill
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm

In [3]:
import elm

Using database version X4-2024-12-31 located in: /home/beyerk/db/exfor/unpack_exfor-2024/X4-2024-12-31


In [4]:
import rxmc

In [5]:
evidence_dir = Path("../evidence/data")
model_dir = Path("../model/")
prior_path = Path("../prior/prior_distribution.pickle")

In [6]:
output_dir = Path("./elm_prior_propagation//")
output_dir.mkdir(parents=True, exist_ok=True)

## read in ELM prior parameter samples

In [7]:
with open(prior_path, "rb") as f:
    prior = pickle.load(f)
prior_samples = prior.rvs(1000)

## read in observations and models

In [8]:
with lzma.open(evidence_dir / "nn_elastic_observations.xz", "rb") as f:
    nn_elastic_obs = pickle.load(f)
with lzma.open(evidence_dir / "pp_elastic_observations.xz", "rb") as f:
    pp_elastic_obs = pickle.load(f)

In [9]:
with open(model_dir / "elm_nn_model.pkl", "rb") as f:
    elm_nn_model = dill.load(f)

In [10]:
with open(model_dir / "elm_pp_model.pkl", "rb") as f:
    elm_pp_model = dill.load(f)

## propagate ELM prior into constraint observables

In [11]:
yth_nn = []
for x in tqdm(nn_elastic_obs):
    yth_nn.append([elm_nn_model(x, *p) for p in prior_samples])
yth_nn = np.hstack(yth_nn)

100%|███████████████████████████████████████████████████████████████| 65/65 [02:28<00:00,  2.28s/it]


In [12]:
yth_pp = []
for x in tqdm(pp_elastic_obs):
    yth_pp.append([elm_pp_model(x, *p) for p in prior_samples])
yth_pp = np.hstack(yth_pp)

100%|█████████████████████████████████████████████████████████████| 160/160 [06:05<00:00,  2.28s/it]


## write elm posterior predictive distributions to disk

In [13]:
with open(output_dir / "nn_elm_posterior_predictive.pkl", "wb") as f:
    pickle.dump(yth_nn, f)

In [14]:
with open(output_dir / "pp_elm_posterior_predictive.pkl", "wb") as f:
    pickle.dump(yth_pp, f)